# Tema 12: NVidia CUDA avanzado

In [ ]:
# Ejecutar en Google Colab
!pip install numpy matplotlib scikit-image numba cython setuptools

### EVITAR ERRORES

!uv pip install -q --system numba-cuda==0.4.0

from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

## A - Warps

In [ ]:
import numpy as np
from numba import cuda

# PARÁMETROS
threads_per_block = 1024
blocks = 1024
n = blocks*threads_per_block # aprox 1M etos

# VECTOR
h_a = np.random.rand(n).astype(np.float32) 
d_a = cuda.to_device(h_a) # copia en GPU

# KERNEL
@cuda.jit
def experiment(a, convergence):

    i = cuda.grid(1) # índice hilo
    block_idx = cuda.blockIdx.x # índice del bloque

    if convergence == True: # CASO CONVERGENCIA
        if (block_idx%4==0): # hilos 0,4,8,... hacen una cosa
            a[i] = a[i]+2 
        elif (block_idx%4==1): # hilos 1,5,9,... hacen otra
            a[i] = np.sin(a[i])
        elif (block_idx%4==2): # hilos 2,6,10,... hacen otra
            a[i] = np.cos(a[i])
        elif (block_idx%4==3): # hilos 3,7,11,... hacen otra
            a[i] = a[i]**0.5

    else: # CASO DIVERGENCIA
        if (i%4==0): 
            a[i] = a[i]+2
        elif (i%4==1):
            a[i] = np.sin(a[i])
        elif (i%4==2):
            a[i] = np.cos(a[i])
        elif (i%4==3):
            a[i] = a[i]**0.5

'''
127	µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each) --> "1 op"
367 µs ± 289 ms per loop (mean ± std. dev. of 7 runs, 1,000 loop each) --> "4 ops"
'''

In [ ]:
%timeit experiment[blocks, threads_per_block](d_a, True); cuda.synchronize()
%timeit experiment[blocks, threads_per_block](d_a, False); cuda.synchronize()

## B - Acceso coalescente a memoria global

In [ ]:
import numpy as np
from numba import cuda

# PARÁMETROS
n = 1024*1024 # nº elementos aprox 1M
threads_per_block = 1024 # hilos por bloque
blocks = 1024 # nº bloques

# Variables en CPU
h_a = np.ones(n).astype(np.float32) 
h_b = h_a.copy().astype(np.float32)

# Variables en GPU
d_a = cuda.to_device(h_a)
d_b = cuda.to_device(h_b)
d_out = cuda.device_array_like(d_a)

# KERNEL
@cuda.jit
def add_experiment(a, b, out, coalesced):
    if coalesced == True: # ACCESO COALESCENTE a MEMORIA GLOBAL
        i = cuda.grid(1)
        # hilos contiguos acceden a posiciones contiguas en memoria
        out[i] = a[i] + b[i] 

    else: # ACCESO NO COALESCENTE a MEMORIA GLOBAL
        thread_idx = cuda.threadIdx.x
        block_idx = cuda.blockIdx.x
        out[thread_idx*threads_per_block+block_idx] = \
            a[thread_idx*threads_per_block+block_idx] + \
            b[thread_idx*threads_per_block+block_idx]

In [ ]:
%timeit add_experiment[blocks, threads_per_block](d_a, d_b, d_out, True); cuda.synchronize()
%timeit add_experiment[blocks, threads_per_block](d_a, d_b, d_out, False); cuda.synchronize()

In [ ]:
from numba import cuda

@cuda.jit
def foo(g): # g es un vector np.float32
    a = 3.14
    i = cuda.grid(1)

    g[i] = a # COALESCENTE: hilos contiguos acceden a posiciones contiguas en memoria

    g[i*2] = a # NO COALESCENTE: hilos contiguos acceden a posiciones no contiguas en memoria
                # ej/ hilo 0 accede a g[0], hilo 1 accede a g[2], hilo 2 accede a g[4], ...

    a = g[i] # COALESCENTE

    a = g[cuda.blockDim.x/2 + i] # COALESCENTE 

    g[i] = a * g[cuda.blockDim.x/2 + i] # COALESCENTE

    g[cuda.blockDim.x - 1 - i] = a # COALESCENTE


__Ejercicio: sumar filas y columnas de matriz__

In [ ]:
import numpy as np
from numba import cuda

@cuda.jit
def row_sums(a, sums, n):
    idx = cuda.grid(1)
    sum = 0.0
    for i in range(n):
        sum += a[idx][i]
    sums[idx] = sum

# TODO: completar
@cuda.jit
def col_sums(a, sums, n):
    None

n = 32768 # matrix side size
threads_per_block = 256
blocks = int(n / threads_per_block)

# Input Matrix
h_a = np.ones(n*n).reshape(n, n).astype(np.float32) 

# Vectors in GPU
d_a = cuda.to_device(h_a)
d_sums = cuda.device_array(shape=(n,), dtype=np.float32)

# Calcular suma de filas
row_sums[blocks, threads_per_block](d_a, d_sums, n)
h_sums = d_sums.copy_to_host()
# Comprobar suma
truth = h_a.sum(axis=1)
np.testing.assert_equal(h_sums,truth)

# TODO: calcular y comprobar suma de columnas

## C - Kernels bidimensionales y tridimensionales

In [ ]:
import numpy as np
from numba import cuda

@cuda.jit
def get_2D_indices(A):
    x, y = cuda.grid(2) # Obtenemos las dos dimensiones
    # Equivalente a:
    # x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    # y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    
    # Escribimos índice x + '.' + índice y
    A[x][y] = x + y / 10

d_A = cuda.device_array(shape=(4,4), dtype=np.float32)
    # Matriz 4x4 en la GPU

blocks = (2, 2) # Grid = 2x2 bloques
threads_per_block = (2, 2) # Bloque = 2x2 threads

np.set_printoptions(precision=1, floatmode="fixed")
get_2D_indices[blocks, threads_per_block](d_A)

print(d_A.copy_to_host())

__Ejercicio: sumar matrices con kernel 2D__

In [ ]:
import numpy as np
from numba import cuda

n = 4096

# TODO: completar
@cuda.jit
def matrix_add(a, b, out, coalesced):
    None

threads_per_block = (32, 32)  # 2D block
blocks = (128, 128) # 2D grid

h_a = np.arange(n*n).reshape(n,n).astype(np.float32)
h_b = h_a.copy().astype(np.float32)

# TODO: copia a la GPU y reserva para la salida

# TODO: invocación kernel y obtención resultados
truth = h_a+h_b
np.testing.assert_equal(h_out, truth)

## D - Memoria compartida

In [ ]:
import numpy as np
from numba import types, cuda

n=5 # nº elementos

# KERNEL para invertir un vector
@cuda.jit
def swap_with_shared(vector, swapped): 
    temp = cuda.shared.array(n, dtype=types.int32) # reserva espacio en memoria compartida (5 enteros de 32 bits)
    idx = cuda.grid(1) # índice hilo
    temp[idx] = vector[idx] # cargar vector de memoria global a memoria compartida

    cuda.syncthreads() # sincronizar hilos (porque luego se leerá la memoria compartida)

    swapped[idx] = temp[n-1 - cuda.threadIdx.x] # copia de memoria compartida a memoria global (INVIRTIENDO VECTOR)
		# swap elements

# VARIABLES EN CPU
h_vector = np.arange(n).astype(np.int32)
print("Antes:",h_vector)
h_swapped = np.zeros_like(h_vector)

# VARIABLES EN GPU
d_vector = cuda.to_device(h_vector)
d_swapped = cuda.device_array(shape=(n,), dtype=np.int32)

# INVOCACIÓN KERNEL
swap_with_shared[1, n](d_vector, d_swapped)
result = d_swapped.copy_to_host()
print("Después:",result)

__Ejercicio: trasponer matriz con coalescencia usando memoria compartida__

In [ ]:
from numba import cuda, types as numba_types
import numpy as np
n = 4096*4096 # 16M

@cuda.jit
def transpose(a, transposed):
    x, y = cuda.grid(2)
    transposed[x][y] = a[y][x]

@cuda.jit
def tile_transpose(a, transposed):
    # Suponer bloques 32x32

    # TODO 1: crear array 32x32 en memoria compartida

    # Calcular índices en array a
    a_row, a_col = cuda.grid(2)
   
    # TODO 2: leer de memoria global y escribir en memoria compartida
    #   los índices locales serán para el buffer en memoria compartida
    #   y los globales para el array a
    
    # TODO 3: Esperamos a que todos los hilos del bloque actualicen la escritura
 
    t_row, t_col = cuda.grid(2)
    # TODO 4: Escribir de memoria compartida (usando índices de hilo)
    #   a memoria global (usando índices de grid) transponiendo cada elemento
    
threads_per_block = (32, 32) # 2D blocks
blocks = (128, 128) #2D grid

# 4096x4096 input and output matrices
h_a = np.arange(n).reshape((4096,4096)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_transposed = cuda.device_array(shape=(4096,4096), dtype=np.float32)

# Invocación a traspose y comprobación
transpose[blocks, threads_per_block](d_a, d_transposed)
result = d_transposed.copy_to_host()
expected = h_a.T
np.testing.assert_equal(result, expected)

# Invocación a tile_traspose y comprobación
tile_transpose[blocks, threads_per_block](d_a, d_transposed)
result = d_transposed.copy_to_host()
expected = h_a.T
np.testing.assert_equal(result, expected)

## F - Comparación accesos a memoria

In [ ]:
from numba import jit, cuda, types as numba_types
import numpy as np

n = 1024*1024 # 1M

# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)

@jit
def transpose_CPU(a, transposed):
    for i in range(1024):
        for j in range(1024):
            transposed[i,j] = a[j,i]

transpose_CPU(h_a, h_out)
expected = h_a.T
np.testing.assert_equal(h_out, expected)

# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_out = cuda.device_array(shape=(1024,1024), dtype=np.float32)

@cuda.jit
def transpose_GPU_1thread(a, transposed):
    for i in range(1024):
        for j in range(1024):
            transposed[i,j] = a[j,i]

transpose_GPU_1thread[1, 1](d_a, d_out)
expected = h_a.T
h_out=d_out.copy_to_host()
np.testing.assert_equal(h_out, expected)

# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_out = cuda.device_array(shape=(1024,1024), dtype=np.float32)

@cuda.jit
def transpose_GPU_1thread_fila(a, transposed):
    i = cuda.threadIdx.x
    for j in range(1024):
        transposed[i,j] = a[j,i]

transpose_GPU_1thread_fila[1, 1024](d_a, d_out)
expected = h_a.T
h_out=d_out.copy_to_host()
np.testing.assert_equal(h_out, expected)

# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_out = cuda.device_array(shape=(1024,1024), dtype=np.float32)

@cuda.jit
def transpose_GPU_1thread_elemento(a, transposed):
    (i,j) = cuda.grid(2)
    transposed[i,j] = a[j,i]

blocks=(32,32)
threads_per_block=(32,32)
transpose_GPU_1thread_elemento[blocks, threads_per_block](d_a, d_out)
h_out=d_out.copy_to_host()
expected = h_a.T
np.testing.assert_equal(h_out, expected)

# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_out = cuda.device_array(shape=(1024,1024), dtype=np.float32)

nthreads=32
nblocks=int(1024/nthreads)
@cuda.jit
def tile_transpose(a, transposed):
    tile = cuda.shared.array((nthreads, nthreads), numba_types.float32)
    a_row, a_col = cuda.grid(2)
    tile[cuda.threadIdx.x, cuda.threadIdx.y] = a[a_row, a_col]
    cuda.syncthreads()
    transposed[a_col, a_row] = tile[cuda.threadIdx.x, cuda.threadIdx.y]

blocks=(nblocks,nblocks)
threads_per_block=(nthreads, nthreads)
tile_transpose[blocks, threads_per_block](d_a, d_out)
h_out=d_out.copy_to_host()
expected = h_a.T
np.testing.assert_equal(h_out, expected)


# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_out = cuda.device_array(shape=(1024,1024), dtype=np.float32)

nthreads=32
nblocks=int(1024/nthreads)
ncols=33
@cuda.jit
def tile_transpose2(a, transposed):
    tile = cuda.shared.array((nthreads, ncols), numba_types.float32)
    a_row, a_col = cuda.grid(2)
    tile[cuda.threadIdx.x, cuda.threadIdx.y] = a[a_row, a_col]
    cuda.syncthreads()
    transposed[a_col, a_row] = tile[cuda.threadIdx.x, cuda.threadIdx.y]

blocks=(nblocks,nblocks)
threads_per_block=(nthreads, nthreads)
tile_transpose[blocks, threads_per_block](d_a, d_out)
h_out=d_out.copy_to_host()
expected = h_a.T
np.testing.assert_equal(h_out, expected)

In [ ]:
%timeit transpose_CPU(h_a, h_out)

In [ ]:
%timeit transpose_GPU_1thread[1, 1](d_a, d_out); cuda.synchronize()

In [ ]:
%timeit transpose_GPU_1thread_fila[1, 1024](d_a, d_out); cuda.synchronize()

In [ ]:
%timeit transpose_GPU_1thread_elemento[(32,32), (32,32)](d_a, d_out); cuda.synchronize()

In [ ]:
%timeit tile_transpose[(32,32), (32,32)](d_a, d_out); cuda.synchronize()
%timeit tile_transpose[(64,64), (16,16)](d_a, d_out); cuda.synchronize()

In [ ]:
%timeit tile_transpose2[(32,32), (32,32)](d_a, d_out); cuda.synchronize()